In [ ]:
#!/usr/bin/env python
__author__ = "Sreenivas Bhattiprolu"
__license__ = "Feel free to copy, I appreciate if you acknowledge Python for Microscopists"

# https://youtu.be/P5FTEryiTl4

"""
OBJECT DETECTION WITH TEMPLATES

Need a source image and a template image.
The template image T is slided over the source image (as in 2D convolution), 
and the program tries to find matches using statistics.
Several comparison methods are implemented in OpenCV.
It returns a grayscale image, where each pixel denotes how much does the 
neighbourhood of that pixel match with template.

Once you got the result, you can use cv2.minMaxLoc() function 
to find where is the maximum/minimum value. Take it as the top-left corner of the 
rectangle and take (w,h) as width and height of the rectangle. 
That rectangle can be drawn on the region of matched template.
"""
# Template matching, single object in an image.
# Multiple methods to see which one works best.

import cv2
import numpy as np
from matplotlib import pyplot as plt

img_gray = cv2.imread('images/level_1.png',0| -1)
template_gray = cv2.imread('images/dart_monkey_no_background.png', 0| -1)
h, w,c = template_gray.shape[::]
h_img,w_img,c_img = img_gray.shape[::]

# methods available: ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
#            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

res = cv2.matchTemplate(img_gray, template_gray, cv2.TM_SQDIFF)
# For TM_SQDIFF, Good match yields minimum value; bad match yields large values
# For all others it is exactly opposite, max value = good fit.

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

top_left = min_loc  #Change to max_loc for all except for TM_SQDIFF
bottom_right = (top_left[0] +w   , top_left[1] + h )
cv2.rectangle(img_gray, top_left, bottom_right, 255, 2)  #White rectangle with thickness 2. 
x = 3




In [ ]:
import cv2
import numpy as np

img = cv2.imread('images/level_1.png')
deltaX = 0
deltaY = 0
cropped_image = img[340+deltaY:530+deltaY, 850+deltaX:1000+deltaX]


img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
template =cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
h, w = template.shape[::] 

#methods available: ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
#            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

res = cv2.matchTemplate(img_gray, template, cv2.TM_SQDIFF_NORMED)
# For TM_SQDIFF, Good match yields minimum value; bad match yields large values
# For all others it is exactly opposite, max value = good fit.

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

top_left = min_loc  #Change to max_loc for all except for TM_SQDIFF
bottom_right = (top_left[0] + w, top_left[1] + h)
cv2.rectangle(img_gray, top_left, bottom_right, 255, 2)  #White rectangle with thickness 2. 

cv2.imshow("Matched image", img_gray)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt


img_with_one_monkey = cv2.imread('images/level_1.png')
deltaX = 0
deltaY = 0
cropped_image = img_with_one_monkey[340+deltaY:530+deltaY, 850+deltaX:1000+deltaX]


img_rgb = cv2.imread('images/multiple_monkeys2.png')

img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

#template =cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
template = cv2.imread('images/dart_monkey_no_background.png',0)

h, w  = template.shape[::] 


res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
plt.imshow(res, cmap='gray')

threshold = 0.5 #Pick only values above 0.8. For TM_CCOEFF_NORMED, larger values = good fit.

loc = np.where( res >= threshold)  
#Outputs 2 arrays. Combine these arrays to get x,y coordinates - take x from one array and y from the other.

#Reminder: ZIP function is an iterator of tuples where first item in each iterator is paired together,
#then the second item and then third, etc. 

for pt in zip(*loc[::-1]):   #-1 to swap the values as we assign x and y coordinate to draw the rectangle. 
    #Draw rectangle around each object. We know the top left (pt), draw rectangle to match the size of the template image.
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)  #Red rectangles with thickness 2. 

#cv2.imwrite('images/template_matched.jpg', img_rgb)
cv2.imshow("Matched image", img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()

In [51]:

# write as a tf or pytorch data generator
from PIL import Image
from PIL.ImageDraw import ImageDraw
import random , uuid , pandas  as pd , numpy as np
from pandas import concat
level = Image.open("./images/empty_level.png").convert("RGBA")
level_with_RGBA = Image.new("RGBA", level.size)
monkey = Image.open("./images/dart_monkey_no_background.png").rotate(180,expand=True) # So monkey points north. 

width_background,height_background = level.size 
width_foreground,height_foreground = monkey.size
outputsize = (np.array([1920,1080])/(120/40)).astype(np.int32)

inframe_x,inframe_y = width_background-width_foreground,height_background -height_foreground
print(inframe_x,inframe_y )

df = pd.DataFrame(columns=["uuid","rotation", "top_left_y","top_left_x", "bottom_right_x","bottom_right_y","class"])


for i in range(2000):

    id = uuid.uuid4().hex

    top_left_x,top_left_y  = [ random.randint(0,inframe_x),random.randint(0,inframe_y) ]

    rotation =  random.randint(0,360)

    foreground_rotated = monkey.rotate(rotation, expand=True)
    
    bottom_right_x,bottom_right_y = np.array( [ top_left_x,top_left_y]) + np.array(foreground_rotated.size)

    center_x = round(top_left_x + foreground_rotated.size[0]/2)
    center_y = round(top_left_y + foreground_rotated.size[1]/2)
    
    

    df2 = pd.DataFrame(
        {"uuid":id,
         "rotation": rotation,
         "top_left_y":top_left_y,
         "top_left_x":top_left_x,
         "bottom_right_x":bottom_right_x,
         "bottom_right_y":bottom_right_y,
         "center_x":center_x,
         "center_y":center_y,
         "class":"dart_monkey"
         },index=["uuid"])
    
    df = concat([df,df2] )

    df.to_csv("./images/data/lookup.csv",header=True,index=False)
    
    level_with_RGBA.paste(level, (0,0), level)
    
    level_with_RGBA.paste(foreground_rotated, (top_left_x,top_left_y), foreground_rotated)

    level_with_RGBA.resize(outputsize).convert("L").save(f"./images/data/{id}.png", optimize=True)

2042 1178
